In [1]:
pip install vncorenlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 40.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645932 sha256=58a8092de672269ec72ce2e34669f1b50534d913888300c89045d057e42379ae
  Stored in directory: /root/.cache/pip/wheels/5d/d9/b3/41f6c6b1ab758561fd4aab55dc0480b9d7a131c6aaa573a3fa
Successfully built vncorenlp
Note: you may need to restart the kernel to use updated packages.


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import re
import string
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import DataLoader

In [3]:
import requests
url = "https://raw.githubusercontent.com/stopwords/vietnamese-stopwords/master/vietnamese-stopwords-dash.txt"
response = requests.get(url)
if response.status_code == 200:
    stop_words = response.text.splitlines()
    print(stop_words[:10])
else:
    print("Failed to fetch the stopwords file.")

['a_lô', 'a_ha', 'ai', 'ai_ai', 'ai_nấy', 'ai_đó', 'alô', 'amen', 'anh', 'anh_ấy']


In [4]:
from vncorenlp import VnCoreNLP

vncorenlp = VnCoreNLP("/kaggle/input/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg,pos,ner,parse", max_heap_size='-Xmx2g')

In [5]:
df = pd.read_csv('/kaggle/input/economic-documents/vietnamese_economic_docs.csv')
df = df.dropna(subset=['text'])

def preprocess_text(text):
    text = re.sub(r'\n+', '. ', text)
    text = re.sub(r'\[\d+\]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\r', '', text)
    text = re.sub(r"Chú thích\.|Xem thêm\.|Liên kết ngoài\.", "", text)
    text = text.strip()
    return text

df['cleaned_text'] = df['text'].apply(preprocess_text)
def tokenize_text(text):
    try:
        tokenized_text = vncorenlp.tokenize(text)
        filtered_sentences = [
            " ".join([word for word in sentence if word not in string.punctuation and word != '–'])
            for sentence in tokenized_text
        ]
        return ". ".join(filtered_sentences)
    except Exception as e:
        print(f"Error tokenizing text {text[:500]} ...: {e}\n")
        return ""
df['tokenized_text'] = df['cleaned_text'].apply(tokenize_text)
corpus = df['tokenized_text'].tolist()
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
words = vectorizer.get_feature_names_out()

df_tfidf = pd.DataFrame(X.T.toarray(), index=words, columns=[f"Doc {i+1}" for i in range(len(corpus))])
df_tfidf

Error tokenizing text Kinh tế Bắc Triều Tiên phản ánh những quan hệ sản xuất, cơ cấu kinh tế và tình hình kinh tế, đời sống tại CHDCND Triều Tiên. Nhìn chung, nền kinh tế CHDCND Triều Tiên là một nền kinh tế công nghiệp với việc sản xuất công nghiệp là hoạt động chính và một nền nông nghiệp gần như tự cung tự cấp do bị cấm vận, đây cũng là một nền kinh tế gần như hoàn toàn thuộc Chính phủ và phát triển theo kế hoạch nhà nước. Quốc gia này có nền kinh tế kế hoạch hóa tập trung cao độ, đất nông nghiệp được tập thể hóa, ...: 400: Unable to parse form content

Error tokenizing text Chủ nghĩa tư bản (Tiếng Anh: capitalism) là một hệ thống kinh tế dựa trên quyền sở hữu tư nhân đối với tư liệu sản xuất và hoạt động sản xuất vì lợi nhuận. Các đặc điểm đặc trưng của chủ nghĩa tư bản bao gồm: tài sản tư nhân, tích lũy tư bản, lao động tiền lương, trao đổi tự nguyện, một hệ thống giá cả và thị trường cạnh tranh. Trong nền kinh tế thị trường tư bản, việc điều hành và đầu tư được quyết định bởi chủ

,Doc 1,Doc 2,Doc 3,Doc 4,Doc 5,Doc 6,Doc 7,Doc 8,Doc 9,Doc 10,...,Doc 1993,Doc 1994,Doc 1995,Doc 1996,Doc 1997,Doc 1998,Doc 1999,Doc 2000,Doc 2001,Doc 2002
00,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.000000,...,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
000,0.007755,0.0,0.0,0.010932,0.0,0.0,0.0,0.0,0.0106,0.044452,...,0.013742,0.027241,0.0,0.01061,0.028694,0.0,0.0,0.0,0.0,0.0
00001,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.000000,...,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
0000120172,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.000000,...,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
00019,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.000000,...,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
에스케이그룹,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.000000,...,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
임금,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.000000,...,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
중화인민공화국_위안,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.000000,...,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0
페소,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0000,0.000000,...,0.000000,0.000000,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,0.0


In [6]:
top_words_list = []

for doc_index in range(len(corpus)):
    important_words = df_tfidf.iloc[:, doc_index]
    vocab_size = (important_words > 0).sum()
    
    N = round(vocab_size * 0.25)
    
    top_important_words = important_words.sort_values(ascending=False).head(N)
    
    rounded_scores = [round(score, 4) for score in top_important_words.values.tolist()]
    
    top_words_list.append({
        "Document": f"Doc {doc_index + 1}",
        "Vocabulary Size": vocab_size,
        "N": N,
        "Top N Words": top_important_words.index.tolist(),
        "TF-IDF Scores": rounded_scores
    })

top_words_df = pd.DataFrame(top_words_list)
top_words_df

,Document,Vocabulary Size,N,Top N Words,TF-IDF Scores
0,Doc 1,421,105,"[đhqg, hcm, đào_tạo, đại_học, trường, luật, kh...","[0.4295, 0.3597, 0.3286, 0.3016, 0.268, 0.1838..."
1,Doc 2,96,24,"[lbo, mua, doanh_nghiệp, leveraged, lại, vay, ...","[0.4033, 0.3629, 0.2648, 0.2551, 0.2197, 0.213..."
2,Doc 3,694,174,"[fed, dự_trữ, liên_bang, ngân_hàng, lãi_suất, ...","[0.4592, 0.3641, 0.3329, 0.3112, 0.1681, 0.166..."
3,Doc 4,804,201,"[lừa_đảo, nạn_nhân, bọn, lừa, séc, nigeria, bị...","[0.4677, 0.3966, 0.3251, 0.2037, 0.195, 0.1737..."
4,Doc 5,162,40,"[công_ty, đại_chúng, phát_hành, chi_phí, publi...","[0.455, 0.3189, 0.3077, 0.1863, 0.1823, 0.1623..."
...,...,...,...,...,...
1997,Doc 1998,186,46,"[sữa, vinamilk, việt_nam, mai_kiều_liên, viii,...","[0.3809, 0.2503, 0.2021, 0.1978, 0.1665, 0.160..."
1998,Doc 1999,267,67,"[khai_hoang, miền, lào_cai, miền_xuôi, núi, ki...","[0.4484, 0.3086, 0.255, 0.2471, 0.2195, 0.1876..."
1999,Doc 2000,41,10,"[win, nguyên_tắc, thắng, đàm_phán, lợi, hợp_tá...","[0.8472, 0.3128, 0.1913, 0.1532, 0.127, 0.1094..."
2000,Doc 2001,234,58,"[madoff, vụ, lừa_đảo, ông, bernard, gian_lận, ...","[0.537, 0.1942, 0.1823, 0.1652, 0.1651, 0.1633..."


In [7]:
labeled_sentences = []

for doc_index in range(len(corpus)):
    document = corpus[doc_index]
    
    sentences = document.split('.')

    for sentence in sentences:
        if sentence == []: continue
        tokens = sentence.split()
        
        top_important_words = top_words_df.iloc[doc_index]["Top N Words"]
        top_important_score = top_words_df.iloc[doc_index]["TF-IDF Scores"]
        
        sentence_labels = []
        for token in tokens:
            token_lower = token.lower()
            if token_lower in stop_words:
                sentence_labels.append(0)
            elif token_lower in top_important_words:
                token_index = top_important_words.index(token_lower)
                sentence_labels.append(top_important_score[token_index])
            else:
                sentence_labels.append(0)
        
        labeled_sentences.append({
            "text": sentence,
            "tokens": tokens,
            "labels_not_process": sentence_labels
        })

labeled_df = pd.DataFrame(labeled_sentences)
labeled_df = labeled_df[labeled_df['tokens'].apply(lambda x: x != [])]
labeled_df

,text,tokens,labels_not_process
0,Trường Đại_học Kinh_tế Luật UEL là trường đại_...,"[Trường, Đại_học, Kinh_tế, Luật, UEL, là, trườ...","[0.268, 0.3016, 0.1457, 0.1838, 0.0907, 0, 0.2..."
1,Tiền_thân của Trường là Khoa_Kinh_tế thuộc Đạ...,"[Tiền_thân, của, Trường, là, Khoa_Kinh_tế, thu...","[0, 0, 0.268, 0, 0.1654, 0, 0.3016, 0, 0.0289,..."
2,Trường đã đạt được nhiều thành_tựu trong đào_...,"[Trường, đã, đạt, được, nhiều, thành_tựu, tron...","[0.268, 0, 0, 0, 0, 0, 0, 0.3286, 0.0804, 0.05..."
5,Lịch_sử hình_thành và phát_triển,"[Lịch_sử, hình_thành, và, phát_triển]","[0, 0, 0, 0.0268]"
6,Ngày 9 tháng 7 năm 1996 theo Quyết_định số 28...,"[Ngày, 9, tháng, 7, năm, 1996, theo, Quyết_địn...","[0, 0, 0, 0, 0, 0, 0, 0.0412, 0, 0, 0, 0, 0, 0..."
...,...,...,...
154515,Cựu sinh_viên Viện Công_nghệ Massachusetts,"[Cựu, sinh_viên, Viện, Công_nghệ, Massachusetts]","[0.1191, 0.1187, 0.0902, 0, 0.0884]"
154516,Cựu sinh_viên Trường Kinh_tế London,"[Cựu, sinh_viên, Trường, Kinh_tế, London]","[0.1191, 0.1187, 0.0778, 0.1855, 0.069]"
154517,Giáo_sư Đại_học Johns_Hopkins,"[Giáo_sư, Đại_học, Johns_Hopkins]","[0.1957, 0.3269, 0]"
154518,Cựu sinh_viên Đại_học British_Columbia,"[Cựu, sinh_viên, Đại_học, British_Columbia]","[0.1191, 0.1187, 0.3269, 0.1039]"


In [8]:
def softmax_with_zeros(x):
    non_zero_values = [xi for xi in x if xi != 0]
    if not non_zero_values: return x
    softmax_values = np.exp(non_zero_values - np.max(non_zero_values))
    softmax_values = softmax_values / softmax_values.sum()
    result = []
    non_zero_idx = 0
    for xi in x:
        if xi != 0:
            result.append(softmax_values[non_zero_idx])
            non_zero_idx += 1
        else:
            result.append(0)
    return result

labeled_df['labels'] = labeled_df['labels_not_process'].apply(lambda x: softmax_with_zeros(x))
labeled_df[['tokens', 'labels_not_process', 'labels']]

,tokens,labels_not_process,labels
0,"[Trường, Đại_học, Kinh_tế, Luật, UEL, là, trườ...","[0.268, 0.3016, 0.1457, 0.1838, 0.0907, 0, 0.2...","[0.057865255733397444, 0.05984256103370221, 0...."
1,"[Tiền_thân, của, Trường, là, Khoa_Kinh_tế, thu...","[0, 0, 0.268, 0, 0.1654, 0, 0.3016, 0, 0.0289,...","[0, 0, 0.2213187690754261, 0, 0.19973751034793..."
2,"[Trường, đã, đạt, được, nhiều, thành_tựu, tron...","[0.268, 0, 0, 0, 0, 0, 0, 0.3286, 0.0804, 0.05...","[0.09433736010336714, 0, 0, 0, 0, 0, 0, 0.1002..."
5,"[Lịch_sử, hình_thành, và, phát_triển]","[0, 0, 0, 0.0268]","[0, 0, 0, 1.0]"
6,"[Ngày, 9, tháng, 7, năm, 1996, theo, Quyết_địn...","[0, 0, 0, 0, 0, 0, 0, 0.0412, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 0, 0, 0, 0.031509680527097496, 0,..."
...,...,...,...
154515,"[Cựu, sinh_viên, Viện, Công_nghệ, Massachusetts]","[0.1191, 0.1187, 0.0902, 0, 0.0884]","[0.25375042169555584, 0.253648941824205, 0.246..."
154516,"[Cựu, sinh_viên, Trường, Kinh_tế, London]","[0.1191, 0.1187, 0.0778, 0.1855, 0.069]","[0.20084637852830858, 0.20076605604246542, 0.1..."
154517,"[Giáo_sư, Đại_học, Johns_Hopkins]","[0.1957, 0.3269, 0]","[0.4672469692208034, 0.5327530307791966, 0]"
154518,"[Cựu, sinh_viên, Đại_học, British_Columbia]","[0.1191, 0.1187, 0.3269, 0.1039]","[0.23722079882821967, 0.2371259294838222, 0.29..."


In [9]:
import torch
from transformers import AutoModel, Trainer, TrainingArguments

class BERTWeighted(nn.Module):
    def __init__(self, pretrained_model_name='vinai/phobert-base-v2'):
        super(BERTWeighted, self).__init__()
        self.bert = AutoModel.from_pretrained(pretrained_model_name)
        self.linear = nn.Linear(self.bert.config.hidden_size, 1)
        
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        embeddings = outputs.last_hidden_state
        word_weights = self.linear(embeddings).squeeze(-1)
        word_weights = word_weights.masked_fill(attention_mask == 0, float('-inf'))
        batch_size, seq_len = attention_mask.shape
        for i in range(batch_size):
            mask_indices = attention_mask[i].nonzero(as_tuple=True)[0]
            first_pos = mask_indices[0]
            last_pos = mask_indices[-1]
            word_weights[i, first_pos] = float('-inf')
            word_weights[i, last_pos] = float('-inf')
        word_weights = torch.nn.functional.softmax(word_weights, dim=-1)
        return word_weights

model = BERTWeighted('vinai/phobert-base-v2')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTWeighted(
  (bert): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

In [10]:
!git clone --single-branch --branch fast_tokenizers_BARTpho_PhoBERT_BERTweet https://github.com/datquocnguyen/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 138580, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 138580 (delta 27), reused 72 (delta 26), pack-reused 138504 (from 1)
Receiving objects: 100% (138580/138580), 160.44 MiB | 30.23 MiB/s, done.
Resolving deltas: 100% (95481/95481), done.


In [11]:
%cd transformers

/kaggle/working/transformers


In [12]:
pip install -e .

Obtaining file:///kaggle/working/transformers
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.9 MB/s eta 0:00:00
  Building editable for transformers (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.32.0.dev0-0.editable-py3-none-any.whl size=38321 sha256=ecb773c5caed45f97e489e15bdcf9da76421b5f29fb9b4a94178a6283257d7d5
  Stored in directory: /tmp/pip-ephem-wheel-cache-quloaqz9/wheels/7e/b2/24/0b3be37b3b423a6f2fd25fd6368a1f4b0888942789c7e68bc6
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Unin

In [13]:
from src.transformers.models.phobert.tokenization_phobert_fast import PhobertTokenizerFast
from datasets import Dataset, DatasetDict
import torch

tokenizer = PhobertTokenizerFast.from_pretrained('vinai/phobert-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
data = []

for index, row in labeled_df.iterrows():
    data.append({
        'tokens': row['tokens'],
        'labels': row['labels']
    })
this_data = pd.DataFrame(data)

dataset = Dataset.from_pandas(this_data)
split_dataset = dataset.train_test_split(test_size=0.2, shuffle=True, seed=42)
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

In [15]:
def tokenize_and_align_labels(row):
    tokenized_inputs = tokenizer(row['tokens'], padding='max_length', max_length=128, truncation=True, is_split_into_words=True)
    word_ids = tokenized_inputs.word_ids()
    
    labels = row['labels']
    aligned_labels = []
    
    for i, word_id in enumerate(word_ids):
        if word_id is None:
            aligned_labels.append(0)
        else:
            aligned_labels.append(labels[word_id])

    if len(aligned_labels) > 128:
        aligned_labels = aligned_labels[:128]
    elif len(aligned_labels) < 128:
        aligned_labels = aligned_labels + [0] * (128 - len(aligned_labels))
    
    tokenized_inputs['labels'] = aligned_labels
    return tokenized_inputs

train_tokenized_dataset = train_dataset.map(tokenize_and_align_labels, batched=False)
train_tokenized_dataset = train_tokenized_dataset.remove_columns(['tokens'])
train_tokenized_dataset

Map:   0%|          | 0/83449 [00:00<?, ? examples/s]

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 83449
})

In [16]:
eval_tokenized_dataset = eval_dataset.map(tokenize_and_align_labels, batched=False)
eval_tokenized_dataset = eval_tokenized_dataset.remove_columns(['tokens'])
eval_tokenized_dataset

Map:   0%|          | 0/20863 [00:00<?, ? examples/s]

Dataset({
    features: ['labels', 'input_ids', 'attention_mask'],
    num_rows: 20863
})

In [17]:
def collate_fn(batch):
    input_ids = torch.tensor([item["input_ids"] for item in batch], dtype=torch.long)
    attention_mask = torch.tensor([item["attention_mask"] for item in batch], dtype=torch.long)
    labels = torch.tensor([item["labels"] for item in batch], dtype=torch.float)
    
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

train_dataloader = DataLoader(train_tokenized_dataset, batch_size=128, collate_fn=collate_fn)
eval_dataloader = DataLoader(eval_tokenized_dataset, batch_size=128, collate_fn=collate_fn)

In [18]:
print("Start training")

model.train()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

try:
    for epoch in range(5):
        total_loss = 0
        for batch in train_dataloader:
            optimizer.zero_grad()
            input_ids = batch["input_ids"].squeeze().to(device)
            attention_mask = batch["attention_mask"].squeeze().to(device)
            labels = batch["labels"].to(device)

            word_weights = model(input_ids, attention_mask)
            epsilon = 1e-8
            word_weights = word_weights + epsilon
            loss = F.kl_div(torch.log(word_weights), labels, reduction='batchmean')

            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        avg_loss = total_loss / len(train_dataloader)
        print(f"Epoch {epoch + 1}, Loss: {avg_loss}")
    
except Exception as e:
    print(f"An error occurred: {e}")

finally:
    torch.save(model.state_dict(), "/kaggle/working/roberta_token_weight.pth")
    print("Model weights saved.")

Start training
Epoch 1, Loss: 0.8622010734640747
Epoch 2, Loss: 0.7812555569446892
Epoch 3, Loss: 0.760066162269174
Epoch 4, Loss: 0.7440788613780875
Epoch 5, Loss: 0.727536555036811
Model weights saved.


In [19]:
model.load_state_dict(torch.load("/kaggle/working/roberta_token_weight.pth"))
model.to(device)
model.eval()
    
total_loss = 0.0
with torch.no_grad():
    for batch in eval_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        
        word_weights = model(input_ids, attention_mask)
        
        loss = F.kl_div(torch.log(word_weights), labels, reduction='batchmean')
        total_loss += loss.item()

avg_loss = total_loss / len(eval_dataloader)
print(f"Evaluation Loss: {avg_loss}")

<ipython-input-19-b48123553559>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/kaggle/working/roberta_token_weight.pth"))


Evaluation Loss: nan


In [20]:
model.load_state_dict(torch.load("/kaggle/working/roberta_token_weight.pth"))
model.to(device)
model.eval()
test_sentences = [
    "Trường đại học này dạy kinh tế rất tốt",
    "Tôi muốn học về quản trị kinh doanh",
    "Kinh tế vĩ mô và vi mô học rất quan trọng trong ngành tài chính",
    "Chính sách tiền tệ có ảnh hưởng trực tiếp đến nền kinh tế quốc gia",
    "Các công ty đa quốc gia ảnh hưởng lớn đến nền kinh tế toàn cầu",
    "Tăng trưởng kinh tế là một chỉ số quan trọng để đo lường sự phát triển của quốc gia",
    "Chúng ta cần có một chiến lược phát triển bền vững trong ngành nông nghiệp",
    "Lạm phát có thể ảnh hưởng nghiêm trọng đến sức mua của người dân",
    "Ngành tài chính cần phải có các biện pháp kiểm soát rủi ro để duy trì sự ổn định",
    "Cải cách thuế là một phần quan trọng trong việc tăng cường hiệu quả nền kinh tế"
]
for test_sentence in test_sentences:
    test_tokens = tokenize_text(test_sentence).split()
    inputs = tokenizer(test_tokens, truncation=True, padding='max_length', max_length=128, return_tensors="pt", is_split_into_words=True)
    inputs = inputs.to(device)

    with torch.no_grad():
        word_weights = model(**inputs)

        tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
        
        print(f"Testing Sentence: {test_sentence}")
        predicted_sum = 0
        for token, weight in zip(tokens, word_weights[0].cpu().numpy()):
            if token != '<pad>':
                print(f"Token: {token}, Prediction: {weight}")
                predicted_sum += weight
        print(f"Sum of Predicted Weights: {predicted_sum}")
        print("\n" + "-"*50 + "\n")

<ipython-input-20-9237754260b7>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/kaggle/working/roberta_token_weight.pth"))


Testing Sentence: Trường đại học này dạy kinh tế rất tốt
Token: <s>, Prediction: 0.0
Token: Trường</w>, Prediction: 0.27580970525741577
Token: đại_học</w>, Prediction: 0.34271469712257385
Token: này</w>, Prediction: 8.674886339576915e-05
Token: dạy</w>, Prediction: 0.0912400409579277
Token: kinh_tế</w>, Prediction: 0.2899472415447235
Token: rất</w>, Prediction: 8.896373037714511e-05
Token: tốt</w>, Prediction: 0.00011257411097176373
Token: </s>, Prediction: 0.0
Sum of Predicted Weights: 0.9999999715873855

--------------------------------------------------

Testing Sentence: Tôi muốn học về quản trị kinh doanh
Token: <s>, Prediction: 0.0
Token: Tôi</w>, Prediction: 0.00025170098524540663
Token: muốn</w>, Prediction: 0.00021832319907844067
Token: học</w>, Prediction: 0.3696726858615875
Token: về</w>, Prediction: 0.0001849628461059183
Token: quản_trị</w>, Prediction: 0.28367823362350464
Token: kinh_doanh</w>, Prediction: 0.3459942042827606
Token: </s>, Prediction: 0.0
Sum of Predicted We